In [1]:
import pandas
import datetime
import feather
import numpy
import scipy.optimize
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.dates import date2num, DateFormatter
%matplotlib inline

In [2]:
# Some constants
eventDate = datetime.datetime(2011,5,16)
timeStart = datetime.datetime(2011,5,16,7,0)
timeEnd = datetime.datetime(2011,5,16,10,0)
sapsRadList = [ "cvw" ]
sapsMlatRange = [ 56., 60. ]
sapsVelCutoff = 50.
sapsNpointsCutoff = 50 # per radar per beam
azimPrcntCntCutoff= 33.
nAzimsCutoff = 5

In [3]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20110516,700,0,2,-2.9119,1.6006,40.1949,-144.9959,16.5659,45.3684,143.7537,40,hok,2011-05-16 07:00:00,-7.4341
1,20110516,700,0,3,-1.7818,2.0959,40.6224,-144.9279,16.5705,45.8067,143.8465,40,hok,2011-05-16 07:00:00,-7.4295
2,20110516,700,0,4,-0.9696,3.3891,41.0394,-144.8684,16.5744,46.2334,143.9308,40,hok,2011-05-16 07:00:00,-7.4256
3,20110516,700,0,5,-0.3545,2.5890,41.4503,-144.8143,16.5781,46.6531,144.0106,40,hok,2011-05-16 07:00:00,-7.4219
4,20110516,700,0,6,0.1277,-0.8005,41.8574,-144.7640,16.5814,47.0682,144.0876,40,hok,2011-05-16 07:00:00,-7.4186


In [4]:
# Filter SAPS scatter using the
# following criteria!!!
# 1) select the radars where SAPS
# is observed
sapsDF = velsDF[ velsDF["radCode"].isin(sapsRadList) ]
# 2) The flows are westward!
# So beams with negative azimuth will have
# negative LoS vels and vice-versa!
sapsDF = sapsDF[sapsDF["azimCalcMag"]/sapsDF["vLos"] > 0. ]
# 3) Set a MLAT limit
sapsDF = sapsDF[ (sapsDF["MLAT"] >= sapsMlatRange[0]) &\
               (sapsDF["MLAT"] <= sapsMlatRange[1]) ]
# 4) Set a velocity cutoff
sapsDF = sapsDF[ numpy.abs(sapsDF["vLos"]) >= sapsVelCutoff ]
# 5) Finally group by radar and beam number to
# discard velocities whose values are
cntPntsSAPSGrp = sapsDF.groupby( ["radCode", "beam"] ).size().reset_index()
# cntPntsSAPSGrp.columns = ["radCode", "beam", "nPoints"]
# sapsDF = pandas.merge( sapsDF, cntPntsSAPSGrp, on=["radCode", "beam"] )
# sapsDF = sapsDF[ sapsDF["nPoints"] >= sapsNpointsCutoff ]

In [5]:
# Some fitting functions
# Fit a sine curve for a given cell
def vel_sine_func(theta, Vmax, delTheta):
    # we are working in degrees but numpy deals with radians
    # convert to radians
    return Vmax * numpy.sin( numpy.deg2rad(theta) +\
                            numpy.deg2rad(delTheta) )

def model_func(theta, Vmax, delTheta):
    vLos = Vmax * numpy.sin( numpy.deg2rad(theta) +\
                            numpy.deg2rad(delTheta) )
    return vLos

initGuess = ( 1000., 10. )

In [6]:
def custom_round(x, base=5):
    return int(base * round(float(x)/base))
# Loop though each time in the data
# to get l-shell fitted data
# store the results in an array
velSaps = []
azimSaps = []
velStd = []
azimStd = []
sapsDates = []
# Get min,median and max of saps MLAT and MLT
minSapsLat = []
medSapsLat = []
maxSapsLat = []
minSapsMLT = []
medSapsMLT = []
maxSapsMLT = []
for sapsTime in sapsDF["date"].unique():
#     print "time---->", sapsTime
    # Get l-shell fitted velocities from
    # the SAPS vLos at different time intervals
    lshellDF = sapsDF[ sapsDF["date"] == sapsTime ]
    # groupby rounded azimuth to 
    # get median vLos to Lshell fit
    lshellDF["azimRnd"] = lshellDF["azimCalcMag"].round()
    selCols = [ "vLos", "MLAT", "normMLT", "azimRnd" ]
    azimDF = lshellDF[selCols].groupby( ["azimRnd"] ).median().reset_index()
    azimDF.columns = [ "azimRnd", "vLos_median", "MLAT_mean", "normMLT_mean" ]
    azimStdDF = lshellDF[selCols].groupby( ["azimRnd"] ).std().reset_index()
    azimStdDF.columns = [ "azimRnd", "vLos_std", "MLAT_std", "normMLT_std" ]
    azimDF = pandas.merge( azimDF, azimStdDF, on="azimRnd" )
    # discard azims with less count
    azimCntDF = lshellDF.groupby("azimRnd").size().reset_index()
    azimCntDF.columns = [ "azimRnd", "nPnts" ]
    azimDF = pandas.merge( azimDF, azimCntDF, on="azimRnd" )
    # discard azimuths where number of points is less than
    # 50% of the maximum (observed across all azimuths)
    azimDF["percCnt"] = azimDF["nPnts"]*100./azimDF["nPnts"].max()
    azimDF = azimDF[ azimDF["percCnt"] >= azimPrcntCntCutoff ]
    # If number of points is less than cutoff
    # discard the fitting process
    if azimDF.shape[0] < nAzimsCutoff:
#         print "discarding fit"
        continue
    # get min,median and max of MLAT, MLT of SAPS scatter
    # MLAT
    minSapsLat.append( lshellDF["MLAT"].min() )
    medSapsLat.append( lshellDF["MLAT"].median() )
    maxSapsLat.append( lshellDF["MLAT"].max() )
    # MLT
    minSapsMLT.append( lshellDF["normMLT"].min() )
    medSapsMLT.append( lshellDF["normMLT"].median() )
    maxSapsMLT.append( lshellDF["normMLT"].max() )
    # Fit the data
    popt, pcov = scipy.optimize.curve_fit(vel_sine_func, \
                        azimDF["azimRnd"].T,\
                        azimDF['vLos_median'].T,
                       p0=initGuess)
    # fitted params
#     print "vMax--->", popt[0],"+/-", pcov[0,0]**0.5
#     print "delTheta--->", popt[1],"+/-", pcov[1,1]**0.5
#     print "--------------------------------------------"
    velSaps.append( popt[0] )
    azimSaps.append( popt[1] )
    velStd.append( pcov[0,0]**0.5 )
    azimStd.append( pcov[1,1]**0.5 )
    sapsDates.append( sapsTime )
    
# convert fit results to a dataframe
fitResultsDF = pandas.DataFrame(
     {'date': sapsDates,
     'velSAPS': velSaps,
     'azimSAPS': azimSaps,
     'velSTD': velStd,
     'azimSTD': azimStd,
     'minLat': minSapsLat,
     'medLat': medSapsLat,
     'maxLat': maxSapsLat,
     'minMLT': minSapsMLT,
     'medMLT': medSapsMLT,
     'maxMLT': maxSapsMLT,
    })
print fitResultsDF

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     azimSAPS    azimSTD                date   maxLat  maxMLT    medLat  \
0   23.435134   2.225455 2011-05-16 07:42:00  58.4999 -0.9530  57.86960   
1   20.697743   1.145668 2011-05-16 07:44:00  58.7361 -0.8378  57.77460   
2   19.864341   0.995761 2011-05-16 07:46:00  59.3614 -0.7618  57.86960   
3   19.768236   0.931059 2011-05-16 07:48:00  59.6174 -0.7280  57.90760   
4   21.456099   0.967506 2011-05-16 07:50:00  59.6174 -0.7372  58.17175   
5   23.594545   4.761864 2011-05-16 07:52:00  59.1999 -0.7854  58.27680   
6   19.276911   7.648839 2011-05-16 07:54:00  58.9694 -0.7522  57.58970   
7   10.056623   7.456939 2011-05-16 07:56:00  58.7361 -0.6371  57.46905   
8   19.890412   5.147700 2011-05-16 07:58:00  58.5404 -0.6036  57.40500   
9   20.410233   3.925630 2011-05-16 08:00:00  58.4214 -0.5599  57.42215   
10  20.427979   3.131936 2011-05-16 08:02:00  59.1016 -0.5264  57.59150   
11  20.029809   3.104352 2011-05-16 08:04:00  59.1999 -0.5324  57.77460   
12  19.922040   2.151380 

In [7]:
# write results to dataframe
feather.write_dataframe(fitResultsDF, '../data/lshell-fits-' +\
                eventDate.strftime("%Y%m%d") + '.feather')